<a href="https://colab.research.google.com/github/hongphucvo/AA/blob/main/scheduling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Student informations

This is the working result of 5 students:

*   Phan Le Tuan Anh: 1811442
*   Tran Khuong Duy:  1811744
*   Thai Phuc Hiep:   1812227
*   Vo Hong Phuc:     1911881
*   Nguyen Tien Dat:  1811884



# Problem Define

Automatic scheduling is the demanding of busy people, who often spend time on making the plan to be efficient. Not only does it help human, but it also solved the problem of industrial company to be more productive.

## PSP (Personal Scheduling Problem):

Assume that in the set of n jobs (***J =  {J1, J2, ... , Jn}***), we have to assign all of them into m window times (***W = {W1, W2, ... , Wm}***). However, these job are splittable, as long as we don't break a jobs into more than ***split_min*** segments. The purpose of solving this problem is to minimize ***Cmax*** - the time all of the jobs had finished.


> PSP: 1|splittable;split_min;available-window|Cmax








## TWSPwJP (Team Work Scheduling Problem):
In PSP, different subjobs of a job can be assigned to multiple different people, which conflict the constraint of consitency for this job. 

The TWSPwJP problem allows every job to be splitted but assigned to only 1 person. 

# Modelling TWSPwJP

## Determine variable


*   x[i,j,t]:  represent the state whether Ji is assigned to Mj at the time Wt (boolean: 0, 1)
*   y[i,j,t]: represent execute time of job/subjob Ji in Mj at Wt, correspond to x[i,j,t] (int: 10, 11, 15, ...)
* s[i,j,t]: start time of this job/subjob Ji in Mj at Wt, correspond to x[i,j,t] (int: 10, 11, 15, ...)
* z[i,j]: represent the state whether Ji is assigned to Mj (boolean: 0, 1)
* v[i1, i2, j, t]: additional variable in linear modelling technique

## Intermediary variable

* c[i,j,t] = s[i,j,t] + y[i,j,t] : finish time of job/subjob Ji in Mj at Wt
* Ci = max (c[i,j,t]) : finish time of job Ji
* Cmax = max (Ci) : finish time of every job in the set J

## Target function:
> min (Cmax)

## Constraint:
### 1. Total 



# Implementation
ádsdasd

## Library Install
In this solution, after modeling the problem, we use these following Python packages and libraries to support:
1. cplex: IBM CPLEX Optimizer API library to create mathematical optimizations.
2. docplex: IBM Decision Optimization CPLEX Modeling for Python.
3. cvxpy: library to convex optimization problems. 
4. json: parsing .json file library
5. numpy: fundamental package for scientific computing

In [ ]:
! pip install cplex docplex

     |████████████████████████████████| 43.3 MB 1.4 MB/s 
     |████████████████████████████████| 610 kB 41.9 MB/s 
  Created wheel for docplex: filename=docplex-2.23.222-py3-none-any.whl size=662847 sha256=a6a433d2dc28fbcdb10b48cd9ec9abf72d0969b2642374c8aab4bc90b46709dd
  Stored in directory: /root/.cache/pip/wheels/a7/c9/fb/cee5a89f304e77a39c466e625ac2830434b76eb8384999d116
Successfully built docplex


In [ ]:
from docplex.mp.model import *
from docplex.cp.model import *
import cvxpy as cp
import json
import numpy as np

## Input dataset
Attribute meanings:


1.   "Jobs": the set of n jobs we have to assign. Each job has 3 attributes:

> 1.1. "Name": The string that specify each job (string: "J1", "J2", ...)

> 1.2. "Processing": The total time to finish this job (int: 1, 5, 10, ...)

> 1.3. "Remaining": The remaining time of this job after some subjobs are assigned (int: 1, 5, 10, ...)

2.   "Machines": the set of m machines/people/workers. Each machines has 2 attributes:

> 2.1. "Name": The string that specify each people (string: "M1", "M2", ...)

> 2.2. "Windows": The set of available windows of this machine. Each window contains:

> > 2.2.1. "Name": Name of the window

> > 2.2.2. "StartTime": Start timestamp of this window

> > 2.2.3. "Capacity": Capacity/the length of this available window

> > 2.2.4. "Remaining": The size of available time in this window

> > 2.2.5. "SubJobs": List of assigned job

3.   "Splitmin": minimum size of a subjob (int: 1, 2, 3, ...)

4.   "Optimal": The ealiest time to finish all jobs (int: 10, 20, 25, ...)

**Test dataset: DS0/input2.json**
> n = 10

> m = 2

In [ ]:
f = open('/content/input2.json')
 
data = json.load(f)
f.close()

In [ ]:
n_jobs = len(data['Jobs'])
J = []
for job in data['Jobs']:
    J += [job['Processing']]

J = np.array(J)

In [ ]:
n_machines = len(data['Machines'])
n_windows = 0

W_cap = []
W_start = []
for i, machine in enumerate(data['Machines']):
    n_windows += len(machine['Windows'])
    for j, window in enumerate(machine['Windows']):
        cap = np.zeros(n_machines)
        cap[i] = window['Capacity']
        W_cap += [cap]
        start = np.array([99999]*n_machines)
        start[i] = window['StartTime']
        W_start += [start]

W_cap = np.array(W_cap)
W_start = np.array(W_start)
W_cap = np.moveaxis(W_cap, 0, -1)
W_start = np.moveaxis(W_start, 0, -1)

In [ ]:
splitmin = data['Splitmin']

## Create model and add constrains

Our models 

In [ ]:
model = CpoModel()

In [ ]:
x = [[[binary_var() for k in range(n_windows)] for j in range(n_machines)] for i in range(n_jobs)]
y = [[[integer_var(min=0, max=J.max()) for k in range(n_windows)] for j in range(n_machines)] for i in range(n_jobs)]
#for i in range(n_jobs):
#    for j in range(n_machines):
#        for k in range(n_windows):
#            model.add(x[i][j][k])
#            model.add(y[i][j][k])
xy = [[[x[i][j][k]*y[i][j][k] for k in range(n_windows)] for j in range(n_machines)] for i in range(n_jobs)]


In [ ]:
# subjob >= splitmin
for i in range(n_jobs):
    for j in range(n_machines):
        for k in range(n_windows):
            model.add_constraint(y[i][j][k] >= splitmin)


In [ ]:
# 1 job chi gan cho 1 manchine
for i in range(n_jobs):
    for j1 in range(n_machines-1):
        x1 = sum([x[i][j1][k] for k in range(n_windows)])
        for j2 in range(j1+1, n_machines):
            x2 = sum([x[i][j2][k] for k in range(n_windows)])
            model.add_constraint(x1*x2 == 0)


In [ ]:
# tong time trong 1 window <= capacity
for j in range(n_machines):
    for k in range(n_windows):
        sum_xy = sum([xy[i][j][k] for i in range(n_jobs)])
        model.add_constraint(sum_xy <= W_cap[j,k])


In [ ]:
# tong time subjob = job
for i in range(n_jobs):
    sum_xy = sum([xy[i][j][k] for j in range(n_machines) for k in range(n_windows)])
    model.add_constraint(sum_xy == J[i])


In [ ]:
# Cmax
C = []
for j in range(n_machines):
    for k in range(n_windows):
        sum_xy = sum([x[i][j][k]*y[i][j][k] for i in range(n_jobs)])
        max_x = max([x[i][j][k] for i in range(n_jobs)])
        c = max_x*(sum_xy+W_start[j,k])
        #sum_xy = sum([x[i][j][k]*y[i][j][k] for i in range(n_jobs)])
        #c = sum_xy+W_start[j,k]
        C += [c]

cmax = max(C)

In [ ]:
model.minimize(cmax)

## Solve

In [ ]:
sol = model.solve(TimeLimit=240)

 ! --------------------------------------------------- CP Optimizer 22.1.0.0 --
 ! Minimization problem - 468 variables, 297 constraints
 ! Presolve      : 36 extractables eliminated
 ! TimeLimit            = 240
 ! Initial process time : 0.01s (0.01s extraction + 0.00s propagation)
 !  . Log search space  : 780.0 (before), 780.0 (after)
 !  . Memory usage      : 671.3 kB (before), 671.3 kB (after)
 ! Using parallel search with 2 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0        468                 -
 + New bound is 0
                        2        252    1   F     0 != _INT_270
 + New bound is 1
 *            45      301  0.05s        1      (gap is 97.78%)
 *            27      543  0.05s        1      (gap is 96.30%)
              27     1000          1    1         5  = _INT_69
              27     1000          1    2         7  = _INT_127
     

In [ ]:
x_result = []
for i in range(n_jobs):
    tmp_x = []
    for j in range(n_machines):
        tmp_x += [np.array([sol[x[i][j][k]] for k in range(n_windows)])]
    tmp_x = np.array(tmp_x)
    x_result += [tmp_x]

x_result = np.array(x_result)

In [ ]:
xy_result = []
for i in range(n_jobs):
    tmp_xy = []
    for j in range(n_machines):
        tmp_xy += [np.array([sol[x[i][j][k]]*sol[y[i][j][k]] for k in range(n_windows)])]
    tmp_xy = np.array(tmp_xy)
    xy_result += [tmp_xy]

xy_result = np.array(xy_result)

In [ ]:
x_result

array([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]],

       [[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],

       [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]])

In [ ]:
xy_result

array([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0]],

       [[0, 5, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 6, 4, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],

       [[4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]])

In [ ]:
W_cap.astype(int)

array([[    7,     5,     8, 99999,     0,     0,     0,     0,     0,
            0,     0,     0,     0],
       [    0,     0,     0,     0,     5,     6,     7,     7, 99999,
            0,     0,     0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            6,     7,     6, 99999]])

# Evaluation

System parameters:
1. GPU: 
2. CPU:
3. Core:
4. Python version:

Model efficiency:
1. #min
2. time
3. %LB

# Future work
Optimize the problem with additional constraint about deadline.
Combine the model with other heuristic methods to optimize performance.